In [11]:
#!/usr/bin/env python
# coding: utf-8
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
import numpy as np
import random
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pandas as pd


# ## Split training/test datasets
input_data = '../model_collocated_10d_filled' 
start_date = '20100101'
end_date = '20221231'

# To reproduce the code
random.seed(56)

# Get all feather files (named by date)
files = [f for f in os.listdir(input_data) if f.endswith('.feather') and os.path.isfile(os.path.join(input_data, f))]

# Get dates (without extension) and shuffle
file_names = [os.path.splitext(f)[0] for f in files]
file_names = [date for date in file_names if date >= start_date]
file_names = [date for date in file_names if date <= end_date]

random.shuffle(file_names)

# Set 0.8 training and 0.2 testing
split_point = int(len(file_names) * 0.8)  

# Split train/test
training_dates = file_names[:split_point]
testing_dates = file_names[split_point:]

# Convert string dates to datetime objects
training_dates = [datetime.strptime(date, '%Y%m%d') for date in training_dates]
testing_dates = [datetime.strptime(date, '%Y%m%d') for date in testing_dates]

n_days_profiles = 5 # How many profiles days to take before and after the central date
chunk_size = 700

model = RandomForestRegressor(n_estimators=40, n_jobs=8, min_samples_leaf = 10, max_depth = 20, verbose=3)

def select_columns_by_prefix(df, prefixes):
    """ Select columns from a DataFrame based on given prefixes. """
    return df[[col for col in df.columns if any(col.startswith(prefix) for prefix in prefixes)]]

# Load data. For each day, we want to get the surrounding days data too. (only profiles)
for i in range(0, len(training_dates), chunk_size):
    model.n_estimators += 10
    dates_to_process = training_dates[i:i+chunk_size]
    print(f'Processing from {i} to {i+chunk_size}. Total of dates is: {len(training_dates)}')
    profiles = []
    for date in dates_to_process:
        date_str = date.strftime('%Y%m%d')
        
        profiles_data = pd.read_feather(f'{input_data}/{date_str}.feather')
        
        if not profiles_data.empty:
            profiles.append(profiles_data) 
            
    if profiles:
        profiles = pd.concat(profiles, ignore_index=True)

    print('Running model fit')
    profiles.dropna(inplace=True)
    profiles = profiles[profiles['LATITUDE'] >= -60]
    profiles = profiles[profiles['LATITUDE'] <= 60]
    predictors_df = profiles.loc[:, ['LATITUDE', 'SSS', 'SST', 'SSH', 'UO', 'VO', 'MLD']]
    #predictors_df = profiles.loc[:, ['SSS', 'SST']]
    target_df = select_columns_by_prefix(profiles, ['ASAL_', 'CTEMP_'])
    model.fit(predictors_df, target_df)
# endloop


import pickle

with open("model_random_forest.pkl", "wb") as f:
    pickle.dump(model, f)

Processing from 0 to 700. Total of dates is: 3798
Running model fit


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50


[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:  1.3min


building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


...

building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100


[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   30.0s


building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66

[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:  3.2min finished


'\nmodel = pickle.load(open("sst_sss_ssh_mld_ssdensity_CTEMP.pkl", \'rb\')) \n\nprofiles = []\nfor date in testing_dates:\n    date_str = date.strftime(\'%Y%m%d\')\n    profiles_data = pd.read_feather(f\'{input_data_10d}/{date_str}.feather\')\n    if not profiles_data.empty:\n        profiles.append(profiles_data) \n            \nif profiles:\n    profiles = pd.concat(profiles, ignore_index=True)  \n\nprofiles.dropna(inplace=True)\n\naccuracy = model.score(profiles[feature_cols], profiles[target_cols])\nprint("Accuracy:", accuracy)\n\n# https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html\n\nimportances = model.feature_importances_\nstd = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)\n\nimport pandas as pd\n\nforest_importances = pd.Series(importances, index=feature_cols)\n\nfig, ax = plt.subplots()\nforest_importances.plot.bar(yerr=std, ax=ax)\nax.set_title("Feature importances using MDI")\nax.set_ylabel("Mean decrease in impu